PARTE 2: Classificação de Imagens Médicas com CNN
CardioIA - A Nova Era da Cardiologia Inteligente

Este notebook implementa dois modelos de classificação:
1. CNN Simples (treinado do zero)
2. Transfer Learning com VGG16 e ResNet50

Inclui:
- Definição e treinamento dos modelos
- Avaliação com métricas (acurácia, matriz de confusão, precisão, recall, F1-score)
- Visualização dos resultados
- Comparação entre os modelos

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json
from pathlib import Path

# TensorFlow e Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Métricas
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve, auc
)

2025-12-07 01:58:11.760012: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-07 01:58:11.760332: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-07 01:58:11.802639: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-07 01:58:12.740126: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation or

In [2]:
# Configurações
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

print("=" * 80)
print("PARTE 2: CLASSIFICAÇÃO DE IMAGENS MÉDICAS COM CNN")
print("CardioIA - A Nova Era da Cardiologia Inteligente")
print("=" * 80)
print(f"\nData de execução: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
print(f"Versão TensorFlow: {tf.__version__}")
print(f"GPU Disponível: {tf.config.list_physical_devices('GPU')}\n")

PARTE 2: CLASSIFICAÇÃO DE IMAGENS MÉDICAS COM CNN
CardioIA - A Nova Era da Cardiologia Inteligente

Data de execução: 07/12/2025 01:58:15
Versão TensorFlow: 2.20.0
GPU Disponível: []



2025-12-07 01:58:15.936688: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [ ]:
# ============================================================================
# SEÇÃO 1: CARREGAMENTO DOS DADOS PRÉ-PROCESSADOS
# ============================================================================
print("\n" + "=" * 80)
print("SEÇÃO 1: CARREGAMENTO DOS DADOS PRÉ-PROCESSADOS")
print("=" * 80)

data_dir = Path('../data/processed')

# Carregar dados
X_train = np.load(str(data_dir / 'X_train.npy'))
X_val = np.load(str(data_dir / 'X_val.npy'))
X_test = np.load(str(data_dir / 'X_test.npy'))
y_train = np.load(str(data_dir / 'y_train.npy'))
y_val = np.load(str(data_dir / 'y_val.npy'))
y_test = np.load(str(data_dir / 'y_test.npy'))

print("\n✓ Dados carregados com sucesso!")
print(f"  X_train shape: {X_train.shape}")
print(f"  X_val shape: {X_val.shape}")
print(f"  X_test shape: {X_test.shape}")

# Adicionar dimensão de canal (grayscale -> 1 canal)
X_train = np.expand_dims(X_train, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

print(f"\n✓ Dimensão de canal adicionada:")
print(f"  X_train shape: {X_train.shape}")
print(f"  X_val shape: {X_val.shape}")
print(f"  X_test shape: {X_test.shape}")

# Número de classes
num_classes = len(np.unique(y_train))
print(f"\n✓ Número de classes: {num_classes}")

# Converter labels para one-hot encoding
y_train_onehot = keras.utils.to_categorical(y_train, num_classes)
y_val_onehot = keras.utils.to_categorical(y_val, num_classes)
y_test_onehot = keras.utils.to_categorical(y_test, num_classes)

print(f"✓ Labels convertidos para one-hot encoding")


SEÇÃO 1: CARREGAMENTO DOS DADOS PRÉ-PROCESSADOS

✓ Dados carregados com sucesso!
  X_train shape: (96, 224, 224)
  X_val shape: (42, 224, 224)
  X_test shape: (60, 224, 224)

✓ Dimensão de canal adicionada:
  X_train shape: (96, 224, 224, 1)
  X_val shape: (42, 224, 224, 1)
  X_test shape: (60, 224, 224, 1)

✓ Número de classes: 3
✓ Labels convertidos para one-hot encoding


In [4]:
# ============================================================================
# SEÇÃO 2: MODELO CNN SIMPLES (TREINADO DO ZERO)
# ============================================================================
print("\n" + "=" * 80)
print("SEÇÃO 2: MODELO CNN SIMPLES (TREINADO DO ZERO)")
print("=" * 80)

def criar_cnn_simples(input_shape, num_classes):
    """
    Cria um modelo CNN simples do zero.
    
    Arquitetura:
    - 2 blocos convolucionais com pooling
    - Camadas densas para classificação
    
    Args:
        input_shape: Forma das imagens de entrada
        num_classes: Número de classes
    
    Returns:
        Modelo Keras compilado
    """
    model = models.Sequential([
        # Bloco 1
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Bloco 2
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Bloco 3
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Camadas Densas
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

# Criar modelo
print("\n✓ Criando modelo CNN simples...")
cnn_model = criar_cnn_simples(input_shape=X_train.shape[1:], num_classes=num_classes)

# Compilar
cnn_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n✓ Modelo compilado!")
print("\nArquitetura do Modelo CNN Simples:")
cnn_model.summary()

# Callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

# Treinar
print("\n✓ Iniciando treinamento do modelo CNN simples...")
print("  (Este processo pode levar alguns minutos)\n")

history_cnn = cnn_model.fit(
    X_train, y_train_onehot,
    validation_data=(X_val, y_val_onehot),
    epochs=50,
    batch_size=16,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

print("\n✓ Treinamento concluído!")


SEÇÃO 2: MODELO CNN SIMPLES (TREINADO DO ZERO)

✓ Criando modelo CNN simples...

✓ Modelo compilado!

Arquitetura do Modelo CNN Simples:


/root/.local/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 224, 224, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 112, 112, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 56, 56, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    25,690,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 26,013,411 (99.23 MB)

 Trainable params: 26,011,747 (99.23 MB)

 Non-trainable params: 1,664 (6.50 KB)


✓ Iniciando treinamento do modelo CNN simples...
  (Este processo pode levar alguns minutos)

Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 675ms/step - accuracy: 0.8333 - loss: 0.6281 - val_accuracy: 0.3333 - val_loss: 1.4736 - learning_rate: 0.0010
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 620ms/step - accuracy: 0.9896 - loss: 0.0352 - val_accuracy: 0.3333 - val_loss: 1.4791 - learning_rate: 0.0010
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 624ms/step - accuracy: 0.9896 - loss: 0.0233 - val_accuracy: 0.3333 - val_loss: 2.4360 - learning_rate: 0.0010
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 636ms/step - accuracy: 1.0000 - loss: 0.0104 - val_accuracy: 0.3333 - val_loss: 3.6476 - learning_rate: 0.0010
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 633ms/step - accuracy: 1.0000 - loss: 0.0082 - val_accuracy: 0.3333 - val_loss: 4.0884 - learning_rate: 0.0010
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 582ms/step - accuracy: 1.0000 - loss: 0.0020
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487

In [5]:
# ============================================================================
# SEÇÃO 3: TRANSFER LEARNING COM VGG16
# ============================================================================
print("\n" + "=" * 80)
print("SEÇÃO 3: TRANSFER LEARNING COM VGG16")
print("=" * 80)

def criar_transfer_learning_vgg16(input_shape, num_classes, congelar_base=True):
    """
    Cria um modelo de Transfer Learning usando VGG16.
    
    Args:
        input_shape: Forma das imagens de entrada
        num_classes: Número de classes
        congelar_base: Se deve congelar os pesos da base pré-treinada
    
    Returns:
        Modelo Keras compilado
    """
    # Carregar modelo pré-treinado (sem as camadas de topo)
    base_model = VGG16(
        weights='imagenet',
        input_shape=input_shape,
        include_top=False
    )
    
    # Congelar camadas da base
    if congelar_base:
        base_model.trainable = False
    
    # Criar novo modelo
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

# Preparar dados para VGG16 (requer 3 canais RGB)
print("\n✓ Preparando dados para VGG16 (convertendo para RGB)...")
X_train_rgb = np.repeat(X_train, 3, axis=-1)
X_val_rgb = np.repeat(X_val, 3, axis=-1)
X_test_rgb = np.repeat(X_test, 3, axis=-1)

print(f"  X_train_rgb shape: {X_train_rgb.shape}")

# Criar modelo VGG16
print("\n✓ Criando modelo Transfer Learning com VGG16...")
vgg_model = criar_transfer_learning_vgg16(
    input_shape=X_train_rgb.shape[1:],
    num_classes=num_classes,
    congelar_base=True
)

# Compilar
vgg_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n✓ Modelo compilado!")
print("\nArquitetura do Modelo Transfer Learning (VGG16):")
vgg_model.summary()

# Treinar
print("\n✓ Iniciando treinamento do modelo VGG16...")
print("  (Este processo pode levar alguns minutos)\n")

history_vgg = vgg_model.fit(
    X_train_rgb, y_train_onehot,
    validation_data=(X_val_rgb, y_val_onehot),
    epochs=50,
    batch_size=16,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

print("\n✓ Treinamento concluído!")


SEÇÃO 3: TRANSFER LEARNING COM VGG16

✓ Preparando dados para VGG16 (convertendo para RGB)...
  X_train_rgb shape: (96, 224, 224, 3)

✓ Criando modelo Transfer Learning com VGG16...

✓ Modelo compilado!

Arquitetura do Modelo Transfer Learning (VGG16):


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,880,835 (56.77 MB)

 Trainable params: 165,379 (646.01 KB)

 Non-trainable params: 14,715,456 (56.14 MB)


✓ Iniciando treinamento do modelo VGG16...
  (Este processo pode levar alguns minutos)

Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 541ms/step - accuracy: 0.3229 - loss: 1.9353 - val_accuracy: 0.3333 - val_loss: 1.1524 - learning_rate: 1.0000e-04
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 502ms/step - accuracy: 0.4375 - loss: 1.6012 - val_accuracy: 0.3333 - val_loss: 1.1426 - learning_rate: 1.0000e-04
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 509ms/step - accuracy: 0.4479 - loss: 1.4129 - val_accuracy: 0.3333 - val_loss: 1.1328 - learning_rate: 1.0000e-04
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 505ms/step - accuracy: 0.4792 - loss: 1.0109 - val_accuracy: 0.3333 - val_loss: 1.1236 - learning_rate: 1.0000e-04
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 509ms/step - accuracy: 0.5729 - loss: 0.9028 - val_accuracy: 0.3333 - val_loss: 1.1153 - learning_rate: 1.0000e-04
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 505ms/step - accuracy: 0.6771 - loss: 0.7800 - val_accuracy: 0.3333 - val_loss: 1.1077 - learning_rate

In [6]:
# ============================================================================
# SEÇÃO 4: TRANSFER LEARNING COM RESNET50
# ============================================================================
print("\n" + "=" * 80)
print("SEÇÃO 4: TRANSFER LEARNING COM RESNET50")
print("=" * 80)

def criar_transfer_learning_resnet50(input_shape, num_classes, congelar_base=True):
    """
    Cria um modelo de Transfer Learning usando ResNet50.
    
    Args:
        input_shape: Forma das imagens de entrada
        num_classes: Número de classes
        congelar_base: Se deve congelar os pesos da base pré-treinada
    
    Returns:
        Modelo Keras compilado
    """
    # Carregar modelo pré-treinado
    base_model = ResNet50(
        weights='imagenet',
        input_shape=input_shape,
        include_top=False
    )
    
    # Congelar camadas da base
    if congelar_base:
        base_model.trainable = False
    
    # Criar novo modelo
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

# Criar modelo ResNet50
print("\n✓ Criando modelo Transfer Learning com ResNet50...")
resnet_model = criar_transfer_learning_resnet50(
    input_shape=X_train_rgb.shape[1:],
    num_classes=num_classes,
    congelar_base=True
)

# Compilar
resnet_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n✓ Modelo compilado!")
print("\nArquitetura do Modelo Transfer Learning (ResNet50):")
resnet_model.summary()

# Treinar
print("\n✓ Iniciando treinamento do modelo ResNet50...")
print("  (Este processo pode levar alguns minutos)\n")

history_resnet = resnet_model.fit(
    X_train_rgb, y_train_onehot,
    validation_data=(X_val_rgb, y_val_onehot),
    epochs=50,
    batch_size=16,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

print("\n✓ Treinamento concluído!")


SEÇÃO 4: TRANSFER LEARNING COM RESNET50

✓ Criando modelo Transfer Learning com ResNet50...

✓ Modelo compilado!

Arquitetura do Modelo Transfer Learning (ResNet50):


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,147,075 (92.11 MB)

 Trainable params: 558,595 (2.13 MB)

 Non-trainable params: 23,588,480 (89.98 MB)


✓ Iniciando treinamento do modelo ResNet50...
  (Este processo pode levar alguns minutos)

Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 488ms/step - accuracy: 0.3542 - loss: 1.1313 - val_accuracy: 0.3333 - val_loss: 1.2217 - learning_rate: 1.0000e-04
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 312ms/step - accuracy: 0.6979 - loss: 0.7923 - val_accuracy: 0.3333 - val_loss: 1.2305 - learning_rate: 1.0000e-04
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 307ms/step - accuracy: 0.8646 - loss: 0.6187 - val_accuracy: 0.3333 - val_loss: 1.2387 - learning_rate: 1.0000e-04
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 311ms/step - accuracy: 0.9375 - loss: 0.4576 - val_accuracy: 0.3333 - val_loss: 1.2471 - learning_rate: 1.0000e-04
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step - accuracy: 0.9564 - loss: 0.4047
Epoch 5: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 314ms/step - accuracy: 0.9583 - loss: 0.4051 - val_accuracy: 0.3333 - val_loss: 1.2531 - learning_rate: 

In [7]:
# ============================================================================
# SEÇÃO 5: AVALIAÇÃO DOS MODELOS
# ============================================================================
print("\n" + "=" * 80)
print("SEÇÃO 5: AVALIAÇÃO DOS MODELOS")
print("=" * 80)

def avaliar_modelo(model, X_test, y_test, y_test_onehot, nome_modelo, usar_rgb=False):
    """
    Avalia um modelo e retorna as métricas.
    
    Args:
        model: Modelo Keras treinado
        X_test: Dados de teste
        y_test: Labels de teste (não one-hot)
        y_test_onehot: Labels de teste (one-hot)
        nome_modelo: Nome do modelo para exibição
        usar_rgb: Se os dados são RGB
    
    Returns:
        Dicionário com métricas
    """
    print(f"\n✓ Avaliando {nome_modelo}...")
    
    # Predições
    y_pred_proba = model.predict(X_test, verbose=0)
    y_pred = np.argmax(y_pred_proba, axis=1)
    
    # Métricas
    acuracia = accuracy_score(y_test, y_pred)
    precisao = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    
    print(f"\n  Métricas de {nome_modelo}:")
    print(f"  • Acurácia:  {acuracia:.4f}")
    print(f"  • Precisão:  {precisao:.4f}")
    print(f"  • Recall:    {recall:.4f}")
    print(f"  • F1-Score:  {f1:.4f}")
    
    # Matriz de confusão
    cm = confusion_matrix(y_test, y_pred)
    
    # Relatório de classificação
    relatorio = classification_report(y_test, y_pred, output_dict=True)
    
    return {
        'nome': nome_modelo,
        'acuracia': acuracia,
        'precisao': precisao,
        'recall': recall,
        'f1_score': f1,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba,
        'matriz_confusao': cm,
        'relatorio': relatorio
    }

# Avaliar todos os modelos
resultados_cnn = avaliar_modelo(cnn_model, X_test, y_test, y_test_onehot, "CNN Simples")
resultados_vgg = avaliar_modelo(vgg_model, X_test_rgb, y_test, y_test_onehot, "VGG16 (Transfer Learning)", usar_rgb=True)
resultados_resnet = avaliar_modelo(resnet_model, X_test_rgb, y_test, y_test_onehot, "ResNet50 (Transfer Learning)", usar_rgb=True)


SEÇÃO 5: AVALIAÇÃO DOS MODELOS

✓ Avaliando CNN Simples...

  Métricas de CNN Simples:
  • Acurácia:  0.3333
  • Precisão:  0.1111
  • Recall:    0.3333
  • F1-Score:  0.1667

✓ Avaliando VGG16 (Transfer Learning)...


/root/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/root/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/root/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



  Métricas de VGG16 (Transfer Learning):
  • Acurácia:  0.3333
  • Precisão:  0.1111
  • Recall:    0.3333
  • F1-Score:  0.1667

✓ Avaliando ResNet50 (Transfer Learning)...


/root/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/root/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/root/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



  Métricas de ResNet50 (Transfer Learning):
  • Acurácia:  0.3333
  • Precisão:  0.1111
  • Recall:    0.3333
  • F1-Score:  0.1667


/root/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/root/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/root/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [9]:
# ============================================================================
# SEÇÃO 6: VISUALIZAÇÃO DOS RESULTADOS
# ============================================================================
print("\n" + "=" * 80)
print("SEÇÃO 6: VISUALIZAÇÃO DOS RESULTADOS")
print("=" * 80)

# Comparação de métricas
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Comparação de Métricas entre Modelos', fontsize=16, fontweight='bold')

modelos = [resultados_cnn, resultados_vgg, resultados_resnet]
nomes = [m['nome'] for m in modelos]
metricas_dict = {
    'Acurácia': [m['acuracia'] for m in modelos],
    'Precisão': [m['precisao'] for m in modelos],
    'Recall': [m['recall'] for m in modelos],
    'F1-Score': [m['f1_score'] for m in modelos]
}

colors = ['#3498db', '#e74c3c', '#f39c12']

for idx, (ax, (metrica, valores)) in enumerate(zip(axes.flatten(), metricas_dict.items())):
    bars = ax.bar(nomes, valores, color=colors)
    ax.set_ylabel(metrica, fontweight='bold')
    ax.set_ylim(0, 1.1)
    ax.set_title(f'{metrica} por Modelo')
    ax.grid(axis='y', alpha=0.3)
    
    # Adicionar valores nas barras
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}',
                ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig('../reports/04_comparacao_metricas.png', dpi=150, bbox_inches='tight')
print("\n✓ Visualização salva: 04_comparacao_metricas.png")
plt.close()

# Matrizes de Confusão
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
fig.suptitle('Matrizes de Confusão', fontsize=14, fontweight='bold')

class_names = ['Normal', 'Cardiomegalia', 'Outras Patologias']

for ax, resultado in zip(axes, modelos):
    cm = resultado['matriz_confusao']
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax, 
                xticklabels=class_names, yticklabels=class_names,
                cbar_kws={'label': 'Contagem'})
    ax.set_title(resultado['nome'])
    ax.set_ylabel('Verdadeiro')
    ax.set_xlabel('Predito')

plt.tight_layout()
plt.savefig('../reports/05_matrizes_confusao.png', dpi=150, bbox_inches='tight')
print("✓ Visualização salva: 05_matrizes_confusao.png")
plt.close()

# Histórico de Treinamento
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
fig.suptitle('Histórico de Treinamento', fontsize=14, fontweight='bold')

historicos = [
    (history_cnn, 'CNN Simples'),
    (history_vgg, 'VGG16'),
    (history_resnet, 'ResNet50')
]

for ax, (history, nome) in zip(axes, historicos):
    ax.plot(history.history['accuracy'], label='Acurácia Treino', linewidth=2)
    ax.plot(history.history['val_accuracy'], label='Acurácia Validação', linewidth=2)
    ax.set_title(f'{nome}')
    ax.set_xlabel('Época')
    ax.set_ylabel('Acurácia')
    ax.legend()
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../reports/06_historico_treinamento.png', dpi=150, bbox_inches='tight')
print("✓ Visualização salva: 06_historico_treinamento.png")
plt.close()


SEÇÃO 6: VISUALIZAÇÃO DOS RESULTADOS

✓ Visualização salva: 04_comparacao_metricas.png
✓ Visualização salva: 05_matrizes_confusao.png
✓ Visualização salva: 06_historico_treinamento.png


In [10]:
# ============================================================================
# SEÇÃO 7: SALVAMENTO DOS MODELOS E RESULTADOS
# ============================================================================
print("\n" + "=" * 80)
print("SEÇÃO 7: SALVAMENTO DOS MODELOS E RESULTADOS")
print("=" * 80)

models_dir = Path('../models')
models_dir.mkdir(parents=True, exist_ok=True)

# Salvar modelos
print("\n✓ Salvando modelos treinados...")
cnn_model.save(str(models_dir / 'cnn_simples.h5'))
vgg_model.save(str(models_dir / 'vgg16_transfer_learning.h5'))
resnet_model.save(str(models_dir / 'resnet50_transfer_learning.h5'))
print("  Modelos salvos com sucesso!")

# Salvar resultados em JSON
resultados_json = {
    'data_execucao': datetime.now().isoformat(),
    'modelos': {
        'CNN Simples': {
            'acuracia': float(resultados_cnn['acuracia']),
            'precisao': float(resultados_cnn['precisao']),
            'recall': float(resultados_cnn['recall']),
            'f1_score': float(resultados_cnn['f1_score'])
        },
        'VGG16': {
            'acuracia': float(resultados_vgg['acuracia']),
            'precisao': float(resultados_vgg['precisao']),
            'recall': float(resultados_vgg['recall']),
            'f1_score': float(resultados_vgg['f1_score'])
        },
        'ResNet50': {
            'acuracia': float(resultados_resnet['acuracia']),
            'precisao': float(resultados_resnet['precisao']),
            'recall': float(resultados_resnet['recall']),
            'f1_score': float(resultados_resnet['f1_score'])
        }
    }
}

with open('../data/resultados_modelos.json', 'w', encoding='utf-8') as f:
    json.dump(resultados_json, f, indent=2, ensure_ascii=False)

print("✓ Resultados salvos em: resultados_modelos.json")


SEÇÃO 7: SALVAMENTO DOS MODELOS E RESULTADOS

✓ Salvando modelos treinados...


  Modelos salvos com sucesso!
✓ Resultados salvos em: resultados_modelos.json


In [11]:
# ============================================================================
# RESUMO FINAL
# ============================================================================
print("\n" + "=" * 80)
print("RESUMO FINAL - PARTE 2")
print("=" * 80)

melhor_modelo = max(modelos, key=lambda x: x['acuracia'])

print(f"""
✓ TREINAMENTO E AVALIAÇÃO CONCLUÍDOS COM SUCESSO!

📊 Resultados dos Modelos:

1. CNN Simples:
   • Acurácia:  {resultados_cnn['acuracia']:.4f}
   • Precisão:  {resultados_cnn['precisao']:.4f}
   • Recall:    {resultados_cnn['recall']:.4f}
   • F1-Score:  {resultados_cnn['f1_score']:.4f}

2. VGG16 (Transfer Learning):
   • Acurácia:  {resultados_vgg['acuracia']:.4f}
   • Precisão:  {resultados_vgg['precisao']:.4f}
   • Recall:    {resultados_vgg['recall']:.4f}
   • F1-Score:  {resultados_vgg['f1_score']:.4f}

3. ResNet50 (Transfer Learning):
   • Acurácia:  {resultados_resnet['acuracia']:.4f}
   • Precisão:  {resultados_resnet['precisao']:.4f}
   • Recall:    {resultados_resnet['recall']:.4f}
   • F1-Score:  {resultados_resnet['f1_score']:.4f}

🏆 Melhor Modelo: {melhor_modelo['nome']} (Acurácia: {melhor_modelo['acuracia']:.4f})

💾 Arquivos Salvos:
   • cnn_simples.h5
   • vgg16_transfer_learning.h5
   • resnet50_transfer_learning.h5
   • resultados_modelos.json

📊 Relatórios Visuais:
   • 04_comparacao_metricas.png
   • 05_matrizes_confusao.png
   • 06_historico_treinamento.png

✅ Próximo passo: Criar protótipo de interface interativa (PARTE 3)
""")

print("=" * 80)
print("Fim da PARTE 2")
print("=" * 80)


RESUMO FINAL - PARTE 2

✓ TREINAMENTO E AVALIAÇÃO CONCLUÍDOS COM SUCESSO!

📊 Resultados dos Modelos:

1. CNN Simples:
   • Acurácia:  0.3333
   • Precisão:  0.1111
   • Recall:    0.3333
   • F1-Score:  0.1667

2. VGG16 (Transfer Learning):
   • Acurácia:  0.3333
   • Precisão:  0.1111
   • Recall:    0.3333
   • F1-Score:  0.1667

3. ResNet50 (Transfer Learning):
   • Acurácia:  0.3333
   • Precisão:  0.1111
   • Recall:    0.3333
   • F1-Score:  0.1667

🏆 Melhor Modelo: CNN Simples (Acurácia: 0.3333)

💾 Arquivos Salvos:
   • cnn_simples.h5
   • vgg16_transfer_learning.h5
   • resnet50_transfer_learning.h5
   • resultados_modelos.json

📊 Relatórios Visuais:
   • 04_comparacao_metricas.png
   • 05_matrizes_confusao.png
   • 06_historico_treinamento.png

✅ Próximo passo: Criar protótipo de interface interativa (PARTE 3)

Fim da PARTE 2
